<a href="https://colab.research.google.com/github/brianpyai/chatbot/blob/main/gemma_playgrounds_GGUF_4Bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install gradio==4.22.0 transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.0 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=

In [ ]:

#@title Built llama-cpp-python with BLAS or CUDA automatically, wait for few minutes .
import os,torch


if torch.cuda.is_available ():
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python' % "-DLLAMA_CUBLAS=on"
else:
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python' % "-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"

os.system (cmd)

0

In [ ]:



from llama_cpp import Llama,LlamaRAMCache,LlamaDiskCache,LlamaTokenizer,LlamaState
from llama_cpp.llama_speculative import LlamaPromptLookupDecoding

import os,torch,json,shlex
from pathlib import Path
from huggingface_hub import hf_hub_download

IN_COLAB = False
Gbase="./generate/"
cache_dir="./hf/"

import torch
if torch.cuda.is_available():
    n_gpu_layers=-1
else :
    n_gpu_layers=0




description ="""<div style="font-family: Arial, sans-serif; padding: 20px;">

This showcases the use of llama-cpp-python to load various versions of 4-bit gemma 2B and 7B models, as well as a simple chat interface created with Gradio. It can be run on a mini computer or even on a phone, with satisfactory results. You are free to modify this code as you wish.
  <h2>🔗My social media links❤️</h2>

Follow <a href="https://www.facebook.com/braiml"  ytarget="_blank">🐍Brian's Page </a> if you want I share more tools .<br>
Follow<a href="https://www.facebook.com/charactersAI" target="_blank">❤️Characters AI</a>
 if you want more  videos .
<br>
  <a href="https://www.facebook.com/brian.pyai" target="_blank"> 📘facebook.com/brian.pyai</a>
 <br>
  <a href="https://www.facebook.com/braiml" target="_blank">🐍Brian's Page </a>
<br>

  <a href="https://www.facebook.com/lovelyai999" target="_blank">🥰AI Hot Shorts </a>



</div>

"""
modelsPath=Gbase +"models/"

modelsPaths=["mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf",
"LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf",
"LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf",
"LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf",
"mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf",
"LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf",
"rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf",
"LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf",
"LoneStriker/zephyr-7b-gemma-sft-v0.1-GGUF/zephyr-7b-gemma-sft-v0.1-Q4_K_M.gguf",
"LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf"]


model_id="ggml-org/gemma-1.1-2b-it-Q8_0-GGUF/gemma-1.1-2b-it.Q8_0.gguf" #@param ["mradermacher/llava-v1.6-mistral-7b-GGUF/llava-v1.6-mistral-7b.Q4_K_M.gguf","billborkowski/llava-NousResearch_Nous-Hermes-2-Vision-GGUF/NousResearch_Nous-Hermes-2-Vision-GGUF_Q4_0.gguf","ggml-org/gemma-1.1-2b-it-Q4_K_M-GGUF/gemma-1.1-2b-it.Q4_K_M.gguf","ggml-org/gemma-1.1-7b-it-Q4_K_M-GGUF/gemma-1.1-7b-it.Q4_K_M.gguf","ggml-org/gemma-1.1-2b-it-Q8_0-GGUF/gemma-1.1-2b-it.Q8_0.gguf","ggml-org/gemma-1.1-7b-it-Q8_0-GGUF/gemma-1.1-7b-it.Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q8_0.gguf","pi-null-mezon/openchat-3.5-0106-gemma-GGUF/ggml-model-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf","LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf","LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf","LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf","rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf","LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf","LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf", "LoneStriker/zephyr-7b-gemma-v0.1-GGUF/zephyr-7b-gemma-v0.1-Q4_K_M.gguf" ,"Lewdiculous/firefly-gemma-7b-GGUF-IQ-Imatrix/firefly-gemma-7b-Q4_K_S-imatrix.gguf"]

def downHgFile(url,targetDir=modelsPath):
    fileName=Path (model_id ).name
    repo =model_id[:-len(fileName)-1]
    fileExists=os.path.exists(os.path.join(targetDir,fileName))

    print(repo,fileName,fileExists )
    if not fileExists:
        print ("Downloading file :")
        hf_hub_download(repo ,filename=fileName,local_dir=targetDir,local_dir_use_symlinks=False)

downHgFile(model_id )
modelPath=os.path.join(modelsPath,Path(model_id).name)

max_tokens=4096 # @param {type:"integer",min:10, max:8192}
model = Llama(modelPath,n_gpu_layers=n_gpu_layers,n_threads=4,max_tokens=4096,logits_all=True,n_ctx=4096,draft_model=LlamaPromptLookupDecoding(num_pred_tokens=10) )

tokenizer = model.tokenize




import gradio as gr
import torch

from threading import Thread




partial_message = ""

def predict(message, history,top_k=300, top_p=0.95, temp=0.85, repeat_penalty=1.1,max_tokens=max_tokens):
    global partial_message
    prompt=f"""
###Below is an instruction that describes my question or task.
Write a response that appropriately completes the request:
{message}
### Response :

"""
    print (message)
    #model_inputs = tokenizer(prompt.encode("utf-8"))
    #generate_kwargs = dict(top_k=top_k, top_p=top_p, temp=temp, repeat_penalty=repeat_penalty)
    generate_kwargs=dict (suffix=None, max_tokens=max_tokens, temperature=temp, top_p=top_p, min_p=0.05, typical_p=1.0, logprobs=None, echo=False, stop=[], frequency_penalty=0.0, presence_penalty=0.0, repeat_penalty=1.1, top_k=top_k, stream=True , seed=None, tfs_z=1.0, mirostat_mode=0, mirostat_tau=5.0, mirostat_eta=0.1, model=None, stopping_criteria=None, logits_processor=None, grammar=None, logit_bias=None)


    partial_message = ""
    outputs =model.create_completion(json.dumps(message,ensure_ascii=True),**generate_kwargs)
    for chunk in outputs:
        #print (chunk )
        content = chunk["choices"][0]["text"]
        #print(content)
        if content:
            partial_message+=content
            yield partial_message
    """
    for new_token in model.generate(model_inputs,**generate_kwargs):
            #print (new_token)
            if new_token in (213,7221,1):break
            s=model.detokenize([new_token])
            #print (s)
            if isinstance(s,bytes):
                try:partial_message += s.decode("utf-8")
                except :break
            yield partial_message
        """

chatbot = gr.Chatbot(likeable=True,show_copy_button=True)
textbox = gr.Textbox (show_copy_button=True)

user="brian" #@param {type:"string"}
password="pwd" #@param {type:"string"}
auth=(user,password)

gr.ChatInterface(predict,chatbot=chatbot,stop_btn="Stop" ,retry_btn="Retry",concurrency_limit=1,description=description,
additional_inputs=[gr.Number(value=100,minimum =0,maximum =1000,precision=0,show_label=True ,label="Topic K"),
gr.Number(value=0.85,minimum =0,maximum =1,precision=3,show_label=True ,label="Topic P"),
gr.Number(value=0.85,minimum =0,maximum =1,precision=3,show_label=True ,label="Temperature"),gr.Number(value=1.1,minimum =1,maximum =3,precision=3,show_label=True ,label="Repeat penalty")],multimodal=True ).launch(debug=True,share=True ,inline=False,auth=auth )

llama_model_loader: loaded meta data with 24 key-value pairs and 164 tensors from ./generate/models/gemma-1.1-2b-it.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          gemma.block_count u32              = 18
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_loader: - kv   7:              gemma.attention.hea

ggml-org/gemma-1.1-2b-it-Q8_0-GGUF gemma-1.1-2b-it.Q8_0.gguf True


llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,256000]  = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,256000]  = [3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  16:                tokenizer.ggml.bos_token_id u32              = 2
llama_model_loader: - kv  17:                tokenizer.ggml.eos_token_id u32              = 1
llama_model_loader: - kv  18:            tokenizer.ggml.unknown_token_id u32              = 3
llama_model_loader: - kv  19:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  20:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  21:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  22:                    tokenizer.chat_template str              = {{ bos_token }}{% if messages[0]['rol...
llama_model_loader: - kv  23: 